In [2]:
import pandas as pd
import requests
from apiKeys import ShoesComKey

# Shop.com API
### API documentation - https://developers.shop.com/externalapis/apn-services/

In [3]:
#Save Locations
SCPath = './Output/ShopComAthletic.csv'

In [4]:
#Get and Check Connection for All Categories
SCCatMain = requests.get('https://api2.shop.com/AffiliatePublisherNetwork/v2/categories?publisherId=FootwearFinder&locale=en_US&site=shop&shipCountry=US&onlyMaProducts=false', headers = ShoesComKey)
SCCatMainRes = SCCatMain.json()
SCCatMain

<Response [200]>

In [5]:
#Verify request limit
SCTestLimit = requests.get('https://api2.shop.com/AffiliatePublisherNetwork/v2/products?publisherId=FootwearFinder&locale=en_US&site=shop&shipCountry=US&start=5001&perPage=50&onlyMaProducts=false', headers = ShoesComKey).json()
SCTestLimit

{'cause': 'please reduce the startIndex value',
 'code': 'startIndex exceeded maxResults',
 'message': "statIndex [5001] exceeded the lucene's resultMax [5000]",
 'status': 400}

In [6]:
#JSON Navigation
print('All Categories:\n---------------------------------------')
for i in SCCatMainRes['categories']: 
    print(i['name'])
print('\nShoes Sub Categories:\n---------------------------------------')
for i in SCCatMainRes['categories'][15]['subCategories']: 
    print(i['name'])
print('\nWomens Sub Categories:\n---------------------------------------')
for i in SCCatMainRes['categories'][15]['subCategories'][0]['subCategories']: 
    print(i['name'])

All Categories:
---------------------------------------
Home Store
Health & Nutrition
Tools
Clothes
Auto
Sports Fan Shop
Beauty
Business
Electronics
Garden
Crafts
Sports and Fitness
Party Supplies
Computers
Shoes
Jewelry
Toys
Movies
Food and Drink
Pet Supplies
Books
Travel
Cameras
Music
Baby
Posters
Video Games
Collectibles
Software

Shoes Sub Categories:
---------------------------------------
Necklaces
Bracelets
Earrings
Rings
Watches
Accessories

Womens Sub Categories:
---------------------------------------
Pendants
Link
Chain
Long


In [7]:
#Limit search in the similar category as Nike products.  Try "Athletic" category.
#Make Dict of Shoe ID, Name, Count

AthleticData = []

#Extract Data
for i in SCCatMainRes['categories']:
    if (i['name'] == 'Shoes'):
        for j in i['subCategories']:
            if (j['name'] == 'Women'):
                for k in j['subCategories']:
                    if (k['name'] == 'Athletic'):
                        AthleticData.append({'id': k['id'],
                                             'name': j['name'],
                                             'productCount': k['productCount']})
            if (j['name'] == 'Men'):
                for k in j['subCategories']:
                    if (k['name'] == 'Athletic'):
                        AthleticData.append({'id': k['id'],
                                             'name': j['name'],
                                             'productCount': k['productCount']})
            if (j['name'] == 'Girls'):
                for k in j['subCategories']:
                    if (k['name'] == 'Athletic'):
                        AthleticData.append({'id': k['id'],
                                             'name': j['name'],
                                             'productCount': k['productCount']})
            if (j['name'] == 'Boys'):
                for k in j['subCategories']:
                    if (k['name'] == 'Athletic'):
                        AthleticData.append({'id': k['id'],
                                             'name': j['name'],
                                             'productCount': k['productCount']})
AthleticData

[{'id': '3-29957_a:12354563', 'name': 'Women', 'productCount': 4122},
 {'id': '3-29953_a:12354562', 'name': 'Men', 'productCount': 3148},
 {'id': '3-29951_a:12354465', 'name': 'Girls', 'productCount': 473},
 {'id': '3-29949_a:12354464', 'name': 'Boys', 'productCount': 434}]

In [8]:
#API documentation indicates 50 products search per page
#Visualize pull request
#Pull and Log

recCount = 1
totalReqs = 0
for record in AthleticData:
    curCount = record['productCount']
    roundCount = 1
    for i in range(1, curCount, 50):
        print(f'Rec: {recCount} Con: {i} Round: {roundCount}')
        roundCount = roundCount + 1
        totalReqs = totalReqs + 1
    print(f'---------------------------------------\nMax: {curCount}\n---------------------------------------')
    recCount = recCount + 1
totalReqs

Rec: 1 Con: 1 Round: 1
Rec: 1 Con: 51 Round: 2
Rec: 1 Con: 101 Round: 3
Rec: 1 Con: 151 Round: 4
Rec: 1 Con: 201 Round: 5
Rec: 1 Con: 251 Round: 6
Rec: 1 Con: 301 Round: 7
Rec: 1 Con: 351 Round: 8
Rec: 1 Con: 401 Round: 9
Rec: 1 Con: 451 Round: 10
Rec: 1 Con: 501 Round: 11
Rec: 1 Con: 551 Round: 12
Rec: 1 Con: 601 Round: 13
Rec: 1 Con: 651 Round: 14
Rec: 1 Con: 701 Round: 15
Rec: 1 Con: 751 Round: 16
Rec: 1 Con: 801 Round: 17
Rec: 1 Con: 851 Round: 18
Rec: 1 Con: 901 Round: 19
Rec: 1 Con: 951 Round: 20
Rec: 1 Con: 1001 Round: 21
Rec: 1 Con: 1051 Round: 22
Rec: 1 Con: 1101 Round: 23
Rec: 1 Con: 1151 Round: 24
Rec: 1 Con: 1201 Round: 25
Rec: 1 Con: 1251 Round: 26
Rec: 1 Con: 1301 Round: 27
Rec: 1 Con: 1351 Round: 28
Rec: 1 Con: 1401 Round: 29
Rec: 1 Con: 1451 Round: 30
Rec: 1 Con: 1501 Round: 31
Rec: 1 Con: 1551 Round: 32
Rec: 1 Con: 1601 Round: 33
Rec: 1 Con: 1651 Round: 34
Rec: 1 Con: 1701 Round: 35
Rec: 1 Con: 1751 Round: 36
Rec: 1 Con: 1801 Round: 37
Rec: 1 Con: 1851 Round: 38
Rec: 1

165

In [10]:
%%time
#Make and Populate DataFrame
SCAthleticDf = pd.DataFrame(columns = ['ProductID', 'Price', 'Name', 'Brand', 'Gender', 'Image', 'URL'])
SCAthleticDf


SCBaseProductUrl = 'https://api2.shop.com/AffiliatePublisherNetwork/v2/products?publisherId=FootwearFinder&locale=en_US&site=shop&shipCountry=US&start='

print('Beginning Product Retrieval')
print('-' * 50)
for record in AthleticData:
    curCatCount = record['productCount']
    curCatID = record['id']
    curCatGen = record['name']
    for pageInc in range(1, curCatCount, 50):
        curProdPageUrl = f'{SCBaseProductUrl}{pageInc}&perPage=50&categoryId={curCatID}&onlyMaProducts=false'
        try:
            curProdPg = requests.get(curProdPageUrl, headers = ShoesComKey).json()
        except:
            print(f'Error Retriving Increment ({pageInc}) For: {curCatID}')
            continue
        for product in curProdPg['products']:
            try:
                prodID = product['id']
            except:
                prodID = ''
            try:
                minPrice = product['minimumPrice']
                maxPrice = product['maximumPrice']
                prodPrice = (minPrice + maxPrice) / 2
            except:
                prodPrice = ''
            try:
                prodName = product['name']
            except:
                prodName = ''
            try:
                prodBrand = product['brand']
            except:
                prodBrand = ''                
            try:
                prodImg = product['image']['sizes'][2]['url']
            except:
                prodImg = ''  
            try:
                prodUrl = product['referralUrl']
            except:
                prodUrl = ''
            ProductData = {'ProductID': prodID,
                          'Price': prodPrice,
                          'Name': prodName,
                          'Brand': prodBrand,
                          'Gender': curCatGen,
                          'Image': prodImg,
                          'URL': prodUrl}
            SCAthleticDf = SCAthleticDf.append(ProductData, ignore_index = True)
    print(f'ID: {curCatID} Successful ({curCatCount})')
SCAthleticDf.set_index('ProductID')
print('-' * 50)
print('Product Retrieval Complete')

Beginning Product Retrieval
--------------------------------------------------
ID: 3-29957_a:12354563 Successful (4122)
ID: 3-29953_a:12354562 Successful (3148)
ID: 3-29951_a:12354465 Successful (473)
ID: 3-29949_a:12354464 Successful (434)
--------------------------------------------------
Product Retrieval Complete
CPU times: user 19.3 s, sys: 418 ms, total: 19.7 s
Wall time: 1min 1s


In [11]:
SCAthleticDf.to_csv(SCPath, index = True)